## Create & Select Columns

#### Create a new variable that contains the full names associated with each stock index symbol.

### SECTION 1: SETUP

1. Import your packages.
2. Use the pandas `set_option` to display all table columns and enable copy on write.
3. Load the `indexes` data set to a variable named `indexes`.

In [11]:
# Import packages used
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [12]:
# Set pandas options to optimize analysis
pd.set_option('display.max_columns', None)
pd.set_option('mode.copy_on_write', True)

# Save the data set to a variable for easy referencing
indexes = pd.read_csv("/Users/kellsworth/Developer/GitHub/python-dabbling/posit-academy/src/files/indexes.csv")

### SECTION 2: WRITE A FUNCTION

Write a function named `recode_symbol()` that will accept a single string value as input and return the associated name:

- DJI: Dow Jones
- FVX: 5-Year Treasury
- GSPC: S&P 500
- IRX: 13-Week Treasury
- IXIC: NASDAQ
- TNX: 10-Year Treasury

In [13]:
# Create function to return stock name in place of symbol
def recode_symbol(symbol):
    if symbol == "^DJI":
        return "Dow Jones"
    elif symbol == "^FVX":
        return "5-Year Treasury"
    elif symbol == "^GSPC":
        return "S&P 500"
    elif symbol == "^IRX":
        return "13-Week Treasury"
    elif symbol == "^IXIC":
        return "NASDAQ"
    elif symbol == "^TNX":
        return "10-Year Treasury"
    else:
        return symbol

Test your function by running the cell below:

In [14]:
recode_symbol('^TNX') # should return `10-Year Treasury`
recode_symbol('^GSPC') # should return `S&P 500`
recode_symbol('^DJI') # should return `Dow Jones`

'Dow Jones'

### SECTION 3: APPLY

Using your function, add a new column to `indexes` called `name` that contains the stock name associated with each symbol.

In [15]:
# Create new column for stock name
indexes['name'] = indexes['symbol'].apply(recode_symbol)

# View updated data frame
indexes

,symbol,date,open,high,low,close,volume,adjusted,name
0,^GSPC,2021-01-04,3764.610107,3769.989990,3662.709961,3700.649902,5015000000,3700.649902,S&P 500
1,^GSPC,2021-01-05,3698.020020,3737.830078,3695.070068,3726.860107,4591020000,3726.860107,S&P 500
2,^GSPC,2021-01-06,3712.199951,3783.040039,3705.340088,3748.139893,6064110000,3748.139893,S&P 500
3,^GSPC,2021-01-07,3764.709961,3811.550049,3764.709961,3803.790039,5099160000,3803.790039,S&P 500
4,^GSPC,2021-01-08,3815.050049,3826.689941,3783.600098,3824.679932,4773040000,3824.679932,S&P 500
...,...,...,...,...,...,...,...,...,...
1507,^TNX,2021-12-27,1.489000,1.494000,1.476000,1.481000,0,1.481000,10-Year Treasury
1508,^TNX,2021-12-28,1.486000,1.489000,1.455000,1.481000,0,1.481000,10-Year Treasury
1509,^TNX,2021-12-29,1.505000,1.558000,1.505000,1.543000,0,1.543000,10-Year Treasury
1510,^TNX,2021-12-30,1.531000,1.550000,1.513000,1.515000,0,1.515000,10-Year Treasury


### SECTION 4: EXTENSION

Expand on your work above by trying something you can learn from a help page or package website.

In [23]:
# Create copy of data frame filtered to only S&P 500
sp500 = indexes.loc[indexes['symbol'] == "^GSPC"]

# View new data frame
sp500

# Calculate 2 week moving average and previous/next values to use in functions
sp500['moving_average'] = sp500['adjusted'].rolling(14).mean()
sp500['previous_price'] = sp500['adjusted'].shift(1)
sp500['next_price'] = sp500['adjusted'].shift(-1)
sp500['previous_average'] = sp500['moving_average'].shift(1)
sp500['next_average'] = sp500['moving_average'].shift(-1)

# Create function to determine trend
def determine_trend(current, previous):
  if current < previous:
    return "downward"
  elif current > previous:
    return "upward"
  else:
    return "constant"
  
# Vectorize trend function
determine_trend_v = np.vectorize(determine_trend)

# Apply function to new column
sp500['price_trend'] = determine_trend_v(sp500['adjusted'], sp500['previous_price'])

# Create function to identify points of support/resistance
def identify_point(avg, prev_avg, next_avg, price, prev_price, next_price):
  if avg <= next_avg and avg <= prev_avg and price < prev_price and price < next_price:
    return "support"
  elif avg >= next_avg and avg >= prev_avg and price > prev_price and price > next_price:
    return "resistance"
  else:
    return ""
  
# Vectorize point function
identify_point_v = np.vectorize(identify_point)

# Apply function to new column
sp500['behavior_shift'] = identify_point_v(sp500['moving_average'], sp500['previous_average'], sp500['next_average'], sp500['adjusted'], sp500['previous_price'], sp500['next_price'])

# Create column to assign value to behavior shift for plotting
sp500['behavior_shift_value'] = np.where(sp500['behavior_shift'] != "", sp500['moving_average'], float('NaN'))

# View updated data frame
sp500['behavior_shift'].unique()

# Set behavior shift colors for use in plot
color_map = {'':'white', 'support':'green', 'resistance':'red'}
behavior_color = sp500['behavior_shift'].map(color_map)

# Create base plot with price trend scatter
fig = px.scatter(
  data_frame=sp500,
  x='date',
  y='adjusted',
  color='price_trend',
  opacity=0.5
)

# Add moving average trend line
fig.add_trace(
  go.Scatter(
    x=sp500['date'],
    y=sp500['moving_average'],
    line=dict(width=0.25),
    mode='lines',
    name="Moving Average Trend"
  )
)

# Add points of behavior shifts
fig.add_trace(
  go.Scatter(
    x=sp500['date'],
    y=sp500['behavior_shift_value'],
    marker=dict(color=behavior_color, symbol='diamond'),
    mode='markers',
    name="Support/Resistance"
  )
)

# Set titles
fig.update_layout(
    title="2021 S&P 500 (GSPC)",
    xaxis_title="Date",
    yaxis_title="Price"
)

# Display plot
fig.show()